In [ ]:
!pip install faiss-cpu sentence-transformers transformers accelerate torch


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
PROMPTS = {
    "base_retrieval_prompt": {
        "role": "You are a helpful Shoplite customer service assistant.",
        "goal": "Provide accurate answers using only the provided Shoplite documentation.",
        "context_guidelines": [
            "Use only information from the provided document snippets",
            "Cite specific documents when possible"
        ],
        "response_format": "Answer: [Your response based on context]\nSources: [List document titles referenced]"
    },
    "complex_question_prompt": {
        "role": "You are a knowledgeable Shoplite support agent.",
        "goal": "Synthesize information from multiple documents to answer complex queries.",
        "context_guidelines": [
            "Combine insights from various documents",
            "Ensure clarity and coherence in the response"
        ],
        "response_format": "Answer: [Your comprehensive response]\nSources: [List all relevant document titles]"
    },
    "clarification_prompt": {
        "role": "You are a Shoplite assistant.",
        "goal": "Ask for clarification when the user's question is ambiguous.",
        "context_guidelines": [
            "Identify unclear aspects of the question",
            "Encourage the user to provide more details"
        ],
        "response_format": "Answer: \"Could you please clarify your question regarding [specific topic]?\"\nSources: []"
    },
    "refusal_prompt": {
        "role": "You are a Shoplite assistant.",
        "goal": "Politely refuse to answer questions that cannot be addressed with the available documentation.",
        "context_guidelines": [
            "Explain the limitation of the knowledge base"
        ],
        "response_format": "Answer: \"I'm sorry, but I cannot provide information on that topic as it is not covered in our documentation.\"\nSources: []"
    }
}


In [ ]:
docs = [
    ("Doc1: How to Sign Up", "Signing up for Shoplite is easy..."),
    ("Doc2: Using the Shopping Cart", "The shopping cart on Shoplite is really useful..."),
    ("Doc3: Paying Safely on Shoplite", "Shoplite offers several ways to pay..."),
    ("Doc4: Tracking Your Orders", "After you place an order on Shoplite..."),
    ("Doc5: Returns and Refunds", "Shoplite understands that sometimes..."),
    ("Doc6: Writing Reviews", "After you buy something on Shoplite..."),
    ("Doc7: Becoming a Seller", "If you want to sell things on Shoplite..."),
    ("Doc8: Managing Inventory", "Shoplite gives sellers tools to manage inventory..."),
    ("Doc9: Seller Fees", "Shoplite charges sellers a fee..."),
    ("Doc10: Getting Help", "If you ever need help on Shoplite..."),
    ("Doc11: The Mobile App", "Shoplite has a mobile app that makes shopping easier..."),
    ("Doc12: Developer Tools", "Shoplite offers an API for developers..."),
    ("Doc13: Security and Privacy", "Shoplite takes security and privacy seriously..."),
    ("Doc14: Promo Codes", "Shoplite often gives out promo codes and discounts..."),
    ("Doc15: Finding Products", "Shoplite makes it easy to find products..."),
    ("Doc16: Giving Feedback", "Shoplite wants to keep getting better, so they ask users for feedback..."),
    ("Doc17: Community Rules and Affiliates", "Shoplite has rules to keep the community safe and friendly...")
]

doc_titles = [d[0] for d in docs]
doc_texts = [d[1] for d in docs]
doc_embeddings = embedder.encode(doc_texts, convert_to_numpy=True)


In [ ]:
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)


In [ ]:
qa_dict = {
    "How do I create a seller account on Shoplite?": "To create a seller account, visit the Shoplite seller registration page, provide business information including tax ID, and complete the verification process which takes 2-3 business days.",
    "What are Shoplite's return policies and how do I track my order status?": "Customers have a 30-day return window for most items. To track your order, log into your account and navigate to the 'Order History' section.",
    "What payment methods are accepted on Shoplite?": "Shoplite accepts various payment methods including credit cards, debit cards, PayPal, and Shoplite gift cards.",
    "How can I apply a promotional code during checkout?": "To apply a promotional code, enter the code in the designated field during the checkout process before finalizing your order.",
    "What features does the Shoplite mobile app offer?": "The Shoplite mobile app allows users to browse products, manage their shopping cart, track orders, and receive notifications about promotions.",
    "How do I return an item purchased on Shoplite?": "To return an item, initiate the return process through your account settings and follow the instructions provided for shipping the item back.",
    "Can I change my shipping address after placing an order?": "You can change your shipping address within 30 minutes of placing your order by contacting customer support.",
    "What is the commission structure for sellers on Shoplite?": "Shoplite charges a commission of 10% on each sale, with additional fees for premium listings.",
    "How do I leave a product review on Shoplite?": "To leave a review, navigate to the product page, click on 'Write a Review', and submit your feedback along with a star rating.",
    "What should I do if I receive a damaged item?": "If you receive a damaged item, contact customer support within 48 hours to initiate a return or exchange.",
    "How can sellers manage their inventory on Shoplite?": "Sellers can manage their inventory through the seller dashboard, where they can add, edit, or remove products as needed.",
    "What are the security measures in place for online transactions?": "Shoplite employs SSL encryption and complies with PCI DSS standards to ensure secure online transactions.",
    "How do I contact Shoplite customer support?": "You can contact Shoplite customer support via email, phone, or live chat available on the website.",
    "Are there any restrictions on promotional codes?": "Yes, promotional codes may have restrictions such as expiration dates and minimum purchase amounts.",
    "How can I track my order delivery status?": "You can track your order delivery status by logging into your account and checking the 'Order History' section for updates.",
    "What types of products can I sell on Shoplite?": "Sellers can list a wide range of products, including electronics, clothing, and home goods, as long as they comply with Shoplite's policies.",
    "How do I reset my Shoplite account password?": "To reset your password, go to the login page, click on 'Forgot Password', and follow the instructions sent to your registered email.",
    "What are the benefits of using the Shoplite mobile app?": "The Shoplite mobile app offers convenience, personalized recommendations, and exclusive app-only discounts.",
    "How do I report a problem with a seller?": "To report a problem with a seller, use the 'Report Seller' feature on their profile page or contact customer support directly.",
    "What should I do if my payment fails during checkout?": "If your payment fails, check your payment details and try again. If the issue persists, contact your bank or Shoplite customer support for assistance."
}

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"  
hf_token = input("Enter your Hugging Face token: ").strip()

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    use_auth_token=hf_token
)


def query_llama(prompt, max_new_tokens=150):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if prompt.strip() in text:
        text = text.replace(prompt, "").strip()
    return text.strip()


In [ ]:
def retrieve_with_scores(query, top_k=3):
    q_emb = embedder.encode([query], convert_to_numpy=True)[0]
    q_norm = np.linalg.norm(q_emb)
    doc_norms = np.linalg.norm(doc_embeddings, axis=1)
    sims = (doc_embeddings @ q_emb) / (q_norm * doc_norms + 1e-10)
    idx = np.argsort(-sims)[:top_k]
    results = [(doc_titles[i], doc_texts[i], float(sims[i])) for i in idx]
    return results, float(np.max(sims))

In [ ]:
SMALLTALK_TRIGGERS = ["hi","hello","hey","how are you","thanks","thank you","bye","goodbye"]

def is_smalltalk(query):
    q = query.lower().strip()
    return any(trig in q for trig in SMALLTALK_TRIGGERS)


In [ ]:
SIMILARITY_THRESHOLD = 0.35
QA_SIM_THRESHOLD = 0.7

def chat():
    print("🤖 Shoplite Chatbot ready! Type 'exit' to quit.\n")
    while True:
        try:
            query = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nGoodbye! 👋")
            break

        if not query:
            continue
        if query.lower() in ["exit","quit"]:
            print("Goodbye! 👋")
            break

        if is_smalltalk(query):
            prompt = f"You are a friendly Shoplite assistant. Respond casually.\nUser: {query}\nAssistant:"
            response = query_llama(prompt, max_new_tokens=60)

            for line in [f"User: {query}", "Assistant:"]:
                if line in response:
                    response = response.replace(line, "").strip()
            print(f"Bot: {response}\n")
            continue

        query_emb = embedder.encode([query], convert_to_numpy=True)[0]
        best_qa = None
        best_score = -1
        for q_text, ans in qa_dict.items():
            q_emb = embedder.encode([q_text], convert_to_numpy=True)[0]
            sim = np.dot(query_emb, q_emb) / (np.linalg.norm(query_emb)*np.linalg.norm(q_emb)+1e-10)
            if sim > best_score:
                best_score = sim
                best_qa = ans
        if best_score >= QA_SIM_THRESHOLD:
            print(f"Bot (QA match): {best_qa}\n")
            continue

        retrieved, best_sim = retrieve_with_scores(query, top_k=3)
        if best_sim < SIMILARITY_THRESHOLD:
            print("Bot: Sorry — that's outside the Shoplite knowledge base.\n")
            continue

        context_snippets = [f"{title} (score={score:.3f}):\n{text}" for title, text, score in retrieved]
        context = "\n\n".join(context_snippets)
        prompt = (
            "You are a helpful Shoplite customer service assistant. Answer using ONLY the context below.\n\n"
            f"Context:\n{context}\n\nUser: {query}\nAssistant:"
        )
        response = query_llama(prompt, max_new_tokens=200)
        sources = ", ".join([t for t,_,_ in retrieved])
        print(f"Bot: {response}\nSources: {sources}\n")

In [ ]:
if __name__ == "__main__":
    chat()